In [1]:
import cv2
import numpy as np

In [2]:
cap = cv2.VideoCapture("get_color.mp4")

In [3]:
def emptyfunc(a):
    pass

cv2.namedWindow("HSV")
cv2.resizeWindow("HSV",640,240)
cv2.createTrackbar("HUE Min","HSV",0,179,emptyfunc)
cv2.createTrackbar("HUE Max","HSV",179,179,emptyfunc)
cv2.createTrackbar("SAT Min","HSV",0,179,emptyfunc)
cv2.createTrackbar("SAT Max","HSV",255,255,emptyfunc)
cv2.createTrackbar("VALUE Min","HSV",0,179,emptyfunc)
cv2.createTrackbar("VALUE Max","HSV",255,255,emptyfunc)

while True:
    ret,getcolor = cap.read()
    if not ret:
        break
    getcolor = cv2.rotate(getcolor, cv2.cv2.ROTATE_90_CLOCKWISE) 
    imgHsv = cv2.cvtColor(getcolor,cv2.COLOR_BGR2HSV)
    
    h_min = cv2.getTrackbarPos("HUE Min","HSV")
    h_max = cv2.getTrackbarPos("HUE Max", "HSV")
    s_min = cv2.getTrackbarPos("SAT Min", "HSV")
    s_max = cv2.getTrackbarPos("SAT Max", "HSV")
    v_min = cv2.getTrackbarPos("VALUE Min", "HSV")
    v_max = cv2.getTrackbarPos("VALUE Max", "HSV")

    lower = np.array([h_min,s_min,v_min])
    upper = np.array([h_max,s_max,v_max])
    mask = cv2.inRange(imgHsv,lower,upper)
    result = cv2.bitwise_and(getcolor,getcolor, mask = mask)

    mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    
    cv2.imshow('Original', getcolor)
    cv2.imshow('Mask', mask)
    cv2.imshow('Result', result)

    if cv2.waitKey(500) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [4]:
print(h_min)
print(h_max)
print(s_min)
print(s_max)
print(v_min)
print(v_max)

89
116
153
255
42
215


In [5]:
background = 0
capn = cv2.VideoCapture("get_bg.mp4")

In [6]:
for i in range(30):
    ret, background = capn.read()
background = np.flip(background, axis=1)
background = cv2.rotate(background, cv2.ROTATE_90_COUNTERCLOCKWISE) 

In [9]:
capture = cv2.VideoCapture("Video.mp4")

In [10]:
while (capture.isOpened()):
    ret, img = capture.read()
    if not ret:
        break
    
    img = np.flip(img, axis=1)
    img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE) 
    
    # Convert the color space from BGR to HSV
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    ## Generat mask to detect color
    
    lower = np.array([h_min,s_min,v_min])
    upper = np.array([h_max,s_max,v_max])
    mask1 = cv2.inRange(hsv, lower, upper)
    
   
    # Open and Dilate the mask image
    kernel = np.ones((5,5), np.uint8)

    mask1 = cv2.morphologyEx(mask1, cv2.MORPH_OPEN,kernel)
    mask1 = cv2.morphologyEx(mask1, cv2.MORPH_DILATE, kernel)

    # Create an inverted mask get the cloth color from the frame
    mask2 = cv2.bitwise_not(mask1)

    # Use the present frame and remove the cloth (segment out) using mask2
    present = cv2.bitwise_and(img, img, mask=mask2)

    # Use the background image to get the segment of the cloth area using mask1
    past = cv2.bitwise_and(background, background, mask=mask1)

    # Generating the final output using the two frames
    finalOutput = cv2.addWeighted(present, 1, past, 1, 0)
    
    cv2.imshow('Original',img)
    cv2.imshow("Final", finalOutput)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()